# Scenario 1

## Summary

**Handling of missing values:**

- Replace non-missing categorical NA values with "None"
- Replace missing `MasVnrArea` values with 0
- Impute missing `LotFrontage` area based on median ratio of `LotArea` to `LotFrontage`
- Exclude column `GarageYrBlt` (which has missing values)
- Exclude rows where the following columns had (1 or 2) missing values: 
    `['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical',
       'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("data/Ames_Housing_Price_Data.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)

## Check Data Shape

In [3]:
df.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,...,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
1,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,...,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
2,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,...,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
3,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,...,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
4,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,...,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal


In [4]:
df.shape

(2580, 81)

## Check Column Types

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2580 entries, 0 to 2579
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PID            2580 non-null   int64  
 1   GrLivArea      2580 non-null   int64  
 2   SalePrice      2580 non-null   int64  
 3   MSSubClass     2580 non-null   int64  
 4   MSZoning       2580 non-null   object 
 5   LotFrontage    2118 non-null   float64
 6   LotArea        2580 non-null   int64  
 7   Street         2580 non-null   object 
 8   Alley          168 non-null    object 
 9   LotShape       2580 non-null   object 
 10  LandContour    2580 non-null   object 
 11  Utilities      2580 non-null   object 
 12  LotConfig      2580 non-null   object 
 13  LandSlope      2580 non-null   object 
 14  Neighborhood   2580 non-null   object 
 15  Condition1     2580 non-null   object 
 16  Condition2     2580 non-null   object 
 17  BldgType       2580 non-null   object 
 18  HouseSty

In [6]:
num_features = 0
cat_features = 0

numerical_columns = []
categorical_columns = []

# Check each column's data type
for column in df.columns:
    dtype = df[column].dtype
    
    # Check if the column is numeric
    if np.issubdtype(dtype, np.number):
        num_features += 1
        numerical_columns.append(column)
    else:
        cat_features += 1
        categorical_columns.append(column)

print(f"Number of numerical features: {num_features}")
print(f"Number of categorical features: {cat_features}")

Number of numerical features: 38
Number of categorical features: 43


## Handle Missing Values

### Cases where NA is not true missing

According to the data description file, NA values in *most* categorical features indicate that that feature is not present in the property.
The exception is `Electrical`, where NA is a true missing value

In [7]:
# Calculate missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values = missing_values.sort_values(ascending=False)

# Calculate percentages
missing_percentage = (missing_values / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Count': missing_values,
    'Percentage': missing_percentage
})
missing_df

,Count,Percentage
PoolQC,2571,99.65
MiscFeature,2483,96.24
Alley,2412,93.49
Fence,2055,79.65
MasVnrType,1573,60.97
FireplaceQu,1241,48.10
LotFrontage,462,17.91
GarageCond,129,5.00
GarageQual,129,5.00
GarageFinish,129,5.00


In [12]:
df[['GarageYrBlt', 'GarageCars', "GarageArea"]].sort_values('GarageCars').head(130)

,GarageYrBlt,GarageCars,GarageArea
2036,NaN,0.0,0.0
429,NaN,0.0,0.0
2017,NaN,0.0,0.0
166,NaN,0.0,0.0
1689,NaN,0.0,0.0
...,...,...,...
2513,NaN,0.0,0.0
2085,NaN,0.0,0.0
690,1939.0,1.0,224.0
1951,1953.0,1.0,616.0


- Replace NaN values where NA indicates "None"

In [31]:
# categorical_columns with NA values
missing_df[missing_df.index.isin(categorical_columns)]

,Count,Percentage
PoolQC,2571,99.65
MiscFeature,2483,96.24
Alley,2412,93.49
Fence,2055,79.65
MasVnrType,1573,60.97
FireplaceQu,1241,48.10
GarageCond,129,5.00
GarageQual,129,5.00
GarageFinish,129,5.00
GarageType,127,4.92


In [32]:
missing_to_replace = [c for c in categorical_columns if c != 'Electrical' ]
missing_to_replace

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [33]:
# replace missing values in categorical columns with "None"
for val in missing_values.index:
    if val in missing_to_replace:
        df[val] = df[val].fillna('None')

In [34]:
# confirm there should be only Electrical column with NA
missing_counts = df[categorical_columns].isna().sum()
missing_counts[missing_counts > 0]

Electrical    1
dtype: int64

In [35]:
missing_df.loc[missing_df.index.isin(numerical_columns)]

,Count,Percentage
LotFrontage,462,17.91
GarageYrBlt,129,5.00
MasVnrArea,14,0.54
BsmtHalfBath,2,0.08
BsmtFullBath,2,0.08
BsmtFinSF1,1,0.04
GarageCars,1,0.04
GarageArea,1,0.04
TotalBsmtSF,1,0.04
BsmtUnfSF,1,0.04


In [36]:
df[df['MasVnrArea'].isna()][['MasVnrType', 'MasVnrArea']]

,MasVnrType,MasVnrArea
175,None,NaN
358,None,NaN
753,None,NaN
787,None,NaN
871,None,NaN
891,None,NaN
951,None,NaN
1041,None,NaN
1161,None,NaN
1355,None,NaN


In [37]:
df['MasVnrArea'].describe().round(1)
# df['MasVnrArea'].hist()

count    2566.0
mean       99.3
std       175.9
min         0.0
25%         0.0
50%         0.0
75%       158.0
max      1600.0
Name: MasVnrArea, dtype: float64

Missing `MasVnrArea` values occur when `MasVnrType` is "None".

=> Set values to 0

In [38]:
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

### Imputations for missing values

- Exclude features with >90% missing values: PoolQC, MiscFeature, Alley

Exclude `GarageYrBlt` column because year built is likely proxy for `GarageQual` and `GarageCond` columns

In [39]:
# To see where GarageYrBlt is missing
df.loc[df['GarageYrBlt'].isna(), ['GarageType', 'GarageYrBlt']]

,GarageType,GarageYrBlt
17,None,NaN
22,None,NaN
44,None,NaN
48,None,NaN
50,None,NaN
...,...,...
2481,None,NaN
2488,None,NaN
2513,None,NaN
2540,None,NaN


In [40]:
df_clean = df.drop('GarageYrBlt', axis=1).copy()

Impute `LotFrontage` area based on median ratio of `LotArea` to `LotFrontage`.

In [41]:
df['LotArea_to_Frontage_Ratio'] = df['LotArea'] / df['LotFrontage']

# median ratio (for the entire dataset)
global_ratio = df['LotArea_to_Frontage_Ratio'].median()
print(f"Median LotArea to LotFrontage ratio: {global_ratio:.2f}")

Median LotArea to LotFrontage ratio: 129.24


In [42]:
missing_mask = df_clean['LotFrontage'].isna()
df_clean.loc[missing_mask, 'LotFrontage'] = df_clean.loc[missing_mask, 'LotArea'] / global_ratio
df_clean['LotFrontage'] = np.round(df_clean['LotFrontage'])

df_clean['LotFrontage'].describe()

count    2580.000000
mean       73.200775
std        47.734548
min        12.000000
25%        59.000000
50%        70.000000
75%        82.000000
max      1274.000000
Name: LotFrontage, dtype: float64

In [43]:
df['LotFrontage'].describe()

count    2118.000000
mean       68.516053
std        22.835831
min        21.000000
25%        57.000000
50%        68.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

- Exclude all other rows containing missing values

In [44]:
# Remove all rows with any missing values (in-place)
df_clean.dropna(inplace=True)

In [45]:
df_clean.isna().any().any()

np.False_

# Save

In [48]:
df_clean.to_csv("data/ames_housing_clean_1.csv", index=False)